In [18]:
!pip install -U transformers datasets evaluate

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"


In [3]:
!pip uninstall -y wandb


Found existing installation: wandb 0.23.0
Uninstalling wandb-0.23.0:
  Successfully uninstalled wandb-0.23.0


In [14]:
sst2 = load_dataset("glue", "sst2")

In [8]:
model_name = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # 0 = negative, 1 = positive
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
for name, param in model.named_parameters():
    print(name, param.shape)

roberta.embeddings.word_embeddings.weight torch.Size([250002, 768])
roberta.embeddings.position_embeddings.weight torch.Size([514, 768])
roberta.embeddings.token_type_embeddings.weight torch.Size([1, 768])
roberta.embeddings.LayerNorm.weight torch.Size([768])
roberta.embeddings.LayerNorm.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.query.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.key.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.value.bias torch.Size([768])
roberta.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.output.dense.bias torch.Size([768])
roberta.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
roberta.encoder.layer.0.attention.output

In [16]:
def preprocess_function(batch):
    return tokenizer(
        batch["sentence"],     # transform each sentences in a numerical format (input_ids, attention_mask)
        padding="max_length",
        truncation=True,
        max_length=128     # sst2 contains small sentences
    )

In [20]:
encoded_sst2 = sst2.map(preprocess_function, batched=True)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Now, we remove the columns idx and sentence because sentence is only useful for the tokenisation.

In [21]:
encoded_sst2 = encoded_sst2.remove_columns(["sentence", "idx"])

In [22]:
encoded_sst2 = encoded_sst2.rename_column("label", "labels")

In [23]:
encoded_sst2.set_format("torch")

In [24]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    return acc

In [11]:
batch_size = 16

training_args = TrainingArguments(
    output_dir="./teacher_en_xlmr_sst2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=0.15,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_sst2["train"],
    eval_dataset=encoded_sst2["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-4194284732.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
trainer.train()


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.223000,0.248062,0.011000,0.923165


TrainOutput(global_step=4210, training_loss=0.2910549317856016, metrics={'train_runtime': 1695.4537, 'train_samples_per_second': 39.723, 'train_steps_per_second': 2.483, 'total_flos': 4430066616860160.0, 'train_loss': 0.2910549317856016, 'epoch': 1.0})

In [31]:
val_results = trainer.evaluate(encoded_sst2["validation"])
print(val_results)


{'eval_loss': 0.2480616420507431, 'eval_model_preparation_time': 0.011, 'eval_accuracy': 0.9231651376146789, 'eval_runtime': 5.6266, 'eval_samples_per_second': 154.977, 'eval_steps_per_second': 9.775, 'epoch': 1.0}


Donc tu ne peux pas faire une évaluation standard dessus.

Dans le benchmark GLUE :

Les labels du test de SST-2 ne sont pas publics.

Ils sont remplacés par des valeurs fictives (-1) pour empêcher l’entraînement après test.

Ce set est prévu pour être soumis à un serveur de compétition, pas pour être utilisé localement.

In [30]:
save_dir = "teacher_en_sst2"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

('teacher_en_sst2/tokenizer_config.json',
 'teacher_en_sst2/special_tokens_map.json',
 'teacher_en_sst2/sentencepiece.bpe.model',
 'teacher_en_sst2/added_tokens.json',
 'teacher_en_sst2/tokenizer.json')